In [13]:
# Benchmark de distintas implementaciones de la medida J (J_measure.py)
# Compara: toro (base), toro2, toro2_1, toro2_2, toro2_2 + joblib (CPU paralelo)
# y toro2_2_torch_batch (PyTorch: CPU/CUDA/MPS).

import J_measure as jm
import importlib
importlib.reload(jm)          # Recarga el módulo por si fue modificado
import numpy as np
from timeit import default_timer as timer

# Tamaño de las series (n) y número de parejas de señales (m)
n = 1000
m = 100

# Datos de prueba: m parejas de series aleatorias de longitud n
a = np.random.rand(n, m)
b = np.random.rand(n, m)

# Arreglos para guardar los tiempos de 100 repeticiones
dt1 = np.zeros(100)  # toro
dt2 = np.zeros(100)  # toro2
dt3 = np.zeros(100)  # toro2_1
dt4 = np.zeros(100)  # toro2_2
dt5 = np.zeros(100)  # toro2_2 en paralelo (joblib)
dt6 = np.zeros(100)  # toro2_2_torch_batch (PyTorch)

# Bucle externo: repetimos el experimento 100 veces para promediar tiempos
for ii in range(100):
    # --- toro (implementación base con ciclos) ---
    tt1 = np.zeros(m)
    start1 = timer()
    for i in range(m):
        tt1[i] = jm.toro(a[:, i], b[:, i], 2, int(n/2))
    dt1[ii] = timer() - start1

    # --- toro2 (NumPy vectorizado con cuadrantes explícitos) ---
    tt2 = np.zeros(m)
    start2 = timer()
    for i in range(m):
        tt2[i] = jm.toro2(a[:, i], b[:, i], 2, int(n/2))
    dt2[ii] = timer() - start2

    # --- toro2_1 (NumPy vectorizado con desplazamiento envuelto) ---
    tt3 = np.zeros(m)
    start3 = timer()
    for i in range(m):
        tt3[i] = jm.toro2_1(a[:, i], b[:, i], 2, int(n/2))
    dt3[ii] = timer() - start3

    # --- toro2_2 (NumPy optimizado con complejos) ---
    tt4 = np.zeros(m)
    start4 = timer()
    for i in range(m):
        tt4[i] = jm.toro2_2(a[:, i], b[:, i], 2, int(n/2))
    dt4[ii] = timer() - start4

    # --- toro2_2 en paralelo por columnas (CPU, joblib.Parallel) ---
    start5 = timer()
    tt5 = jm.toro2_2_joblib_batch(a, b, 2, int(n/2))
    dt5[ii] = timer() - start5

    # --- toro2_2_torch_batch (PyTorch, batch en CPU/CUDA/MPS) ---
    start6 = timer()
    tt6 = jm.toro2_2_torch_batch(a, b, 2, n//2)  # devuelve un tensor de tamaño m
    dt6[ii] = timer() - start6

# Resumen de rendimiento relativo respecto a la implementación base toro
print('tiempo promedio en analizar', m, 'parejas de', n, 'datos, repetido 100 veces con toro:',np.mean(dt1))
print('toro2 es en promedio (100 rep.)',np.mean(dt1)/np.mean(dt2), 'veces más rápido que toro')
print('toro2_1 es en promedio (100 rep.)',np.mean(dt1)/np.mean(dt3), 'veces más rápido que toro')
print('toro2_2 es en promedio (100 rep.)',np.mean(dt1)/np.mean(dt4), 'veces más rápido que toro')
print('toro2_2 (joblib) es en promedio (100 rep.)', np.mean(dt1)/np.mean(dt5), 'veces más rápido que toro')
print('toro2_2_torch_batch es en promedio (100 rep.)',np.mean(dt1)/np.mean(dt6), 'veces más rápido que toro')

tiempo promedio en analizar 100 parejas de 1000 datos, repetido 100 veces con toro: 1.39074410500034
toro2 es en promedio (100 rep.) 50.687595869845104 veces más rápido que toro
toro2_1 es en promedio (100 rep.) 82.85833574748703 veces más rápido que toro
toro2_2 es en promedio (100 rep.) 100.11697376237038 veces más rápido que toro
toro2_2 (joblib) es en promedio (100 rep.) 32.650651201089886 veces más rápido que toro
toro2_2_torch_batch es en promedio (100 rep.) 445.69603226301575 veces más rápido que toro
